In [8]:
from pyquil import Program, get_qc
from pyquil.api import WavefunctionSimulator
from pyquil.gates import *
import numpy as np

Let's create some random wavefunction that we want to teleport.

In [17]:
def random_unitary():
    """
    :return: array of shape (2, 2) representing random unitary matrix drawn from Haar measure
    """
    # draw complex matrix from Ginibre ensemble
    z = np.random.randn(2, 2) + 1j * np.random.randn(2, 2)
    # QR decompose this complex matrix
    q, r = np.linalg.qr(z)
    # make this decomposition unique
    d = np.diagonal(r)
    l = np.diag(d) / np.abs(d)
    return np.matmul(q, l)


def random_wavefunc():
    """
    :return: Program for a quantum circuit creating a random 1-qubit state
    """
    p = Program()
    p.defgate("RandomUnitary", random_unitary())
    p.inst(("RandomUnitary", 2))
    return p

r = random_unitary()
print(r)

[[-0.67750594-0.12253513j -0.20330951+0.69615809j]
 [ 0.71625289+0.11380963j -0.17885734+0.6648603j ]]


In [18]:
wfn_sim = WavefunctionSimulator()

In [19]:
prog = Program()
alice_regs = prog.declare('alice_regs', 'BIT', 2)

In [5]:
prog += random_wavefunc()
wfn_orig = wfn_sim.wavefunction(prog)
print(wfn_orig)

(-0.1094272972-0.6654493347j)|0> + (-0.3459148898-0.6523386686j)|1>


In [7]:
print(Program.__doc__)

A list of pyQuil instructions that comprise a quantum program.

    >>> from pyquil import Program
    >>> from pyquil.gates import *
    >>> p = Program()
    >>> p += H(0)
    >>> p += CNOT(0, 1)
    


Suppose we created the above wavefunction at some location A. We would like to teleport this wavefunction to some distant location B. The teleportation protocal necessitates the observers at A and B (Alice and Bob, respectively) to share a Bell state as a resource. So let's add the creation of a Bell state to our Program.

In [ ]:
def bell_state(q0, q1):
    """
    :param int q0: first (R-to-L) qubit to form part of the Bell state
    :param int q1: second (R-to-L) qubit to form part of the Bell state
    :return: Program creating a Bell state between input qubits
    """
    prog = Program()
    

In [ ]:
# Note that the random wavefunction was created over qubit 2
# before the creation of a Bell state over qubits 1 and 0
prog += bell_state(0, 1)

So now we've created a 3 qubit state $\vert \psi \rangle \vert \beta_{00} \rangle$, which can be re-written as $\frac{1}{2} \left( \vert \beta_{00} \rangle \vert \psi \rangle + \vert \beta_{01} \rangle (X \vert \psi \rangle) + \vert \beta_{10} \rangle (Z \vert \psi \rangle) + \vert \beta_{11} \rangle (XZ\vert \psi \rangle) \right)$.<br>
<br>
The last two qubits (R-to-L) are in Alice's possession. The teleportation protocol requires Alice to measure these qubits in the Bell basis. Let's make sure she can do this.

In [ ]:
def bell_basis_circuit(q0, q1):
    """
    :param int q0: first (R-to-L) qubit that Alice will measure
    :param int q1: second (R-to-L) qubit that Alice will measure
    :return: Program preparing a measurement in the Bell basis
    """
    # TODO

In [ ]:
# Prepare the measurement of qubits 2 and 1 in the Bell basis
prog += bell_basis_circuit(1, 2)

In order for Bob to successfully reconstruct the wavefunction $\vert \psi \rangle$, he needs to receive the two classical bits from Alice corresponding to her measurement of the two qubits in her posession in the Bell basis. He then needs to conditionally apply quantum gate(s) to his qubit based on the classical bits he receives from Alice, according to:<br><br>
$M_1, M_2 \quad\quad\quad \text{Bob performs}$<br>
$\,\,0,0 \quad\quad\quad\quad\quad I$<br>
$\,\,0,1 \quad\quad\quad\quad\quad X$<br>
$\,\,1,0 \quad\quad\quad\quad\quad Z$<br>
$\,\,1,1 \quad\quad\quad\quad\quad Z\cdot X$<br>
<br>
Let's write a function that allows Bob to do this.

In [ ]:
def conditionally_apply_gate(p, q0, q1, q2, alice_regs):
    """
    NOTE: This function directly modifies the input Program,
        but does not return a new Program
    
    :param p: Program that performs the teleportation protocol
        upto conditional application of Bob's gate(s)
    :param int q0: only qubit that Bob possesses
    :param int q1: first (R-to-L) qubit that Alice measures
    :param int q2: second (R-to-L) qubit that Alice measures
    :param list alice_regs: classical registers holding Alice's
        measurements of her qubits
    """
    # TODO

Apply these conditional gates on the wavefunction

In [ ]:
conditionally_apply_gate(prog, 0, 1, 2, alice_regs)

In the end, the wavefunction we have produced over all 3 qubits looks like:

In [ ]:
wfn = wfn_sim.wavefunction(prog)
print (wfn)

Bob has possession of the right-most qubit. Let's compare this wavefunction to the one Alice originally posssesed. Note that the wavefunction above can be decomposed as a product state of qubit 0 and qubits 1 and 2: $\vert \psi \rangle = \vert \psi \rangle_{21} \otimes \vert \psi \rangle_{0}$.

In [ ]:
print (wfn_orig)

In [ ]:
# Make sure wavefunction has indeed been teleported

np.testing.assert_almost_equal(
    np.sum([v for k, v in wfn.get_outcome_probs().items()
            if k[-1] == '0']), 
    wfn_orig.get_outcome_probs()['0'])

np.testing.assert_almost_equal(
    np.sum([v for k, v in wfn.get_outcome_probs().items()
            if k[-1] == '1']), 
    wfn_orig.get_outcome_probs()['1'])